## MOVIE RECOMMDER SYSTEM

In [1]:
## load libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import difflib

In [2]:
#3 load datas

netflix = pd.read_csv('C:/movierecommender/notebook/data/netflix_titles.csv')
amazon =  pd.read_csv('C:/movierecommender/notebook/data/amazon_prime_titles.csv')
hulu =  pd.read_csv('C:/movierecommender/notebook/data/hulu_titles.csv')
disney =  pd.read_csv('C:/movierecommender/notebook/data/disney_plus_titles.csv')

In [3]:
netflix.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [4]:
amazon.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,NaN,69 min,Documentary,"Pink breaks the mold once again, bringing her ..."
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,NaN,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...


In [5]:
hulu.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Ricky Velez: Here's Everything,NaN,NaN,NaN,"October 24, 2021",2021,TV-MA,NaN,"Comedy, Stand Up",​Comedian Ricky Velez bares it all with his ho...
1,s2,Movie,Silent Night,NaN,NaN,NaN,"October 23, 2021",2020,NaN,94 min,"Crime, Drama, Thriller","Mark, a low end South London hitman recently r..."
2,s3,Movie,The Marksman,NaN,NaN,NaN,"October 23, 2021",2021,PG-13,108 min,"Action, Thriller",A hardened Arizona rancher tries to protect an...
3,s4,Movie,Gaia,NaN,NaN,NaN,"October 22, 2021",2021,R,97 min,Horror,A forest ranger and two survivalists with a cu...
4,s5,Movie,Settlers,NaN,NaN,NaN,"October 22, 2021",2021,NaN,104 min,"Science Fiction, Thriller",Mankind's earliest settlers on the Martian fro...


In [6]:
disney.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso Ramirez Ramos, Dave Wasson","Chris Diamantopoulos, Tony Anselmo, Tress MacN...",NaN,"November 26, 2021",2016,TV-G,23 min,"Animation, Family",Join Mickey and the gang as they duck the halls!
1,s2,Movie,Ernest Saves Christmas,John Cherry,"Jim Varney, Noelle Parker, Douglas Seale",NaN,"November 26, 2021",1988,PG,91 min,Comedy,Santa Claus passes his magic bag to a new St. ...
2,s3,Movie,Ice Age: A Mammoth Christmas,Karen Disher,"Raymond Albert Romano, John Leguizamo, Denis L...",United States,"November 26, 2021",2011,TV-G,23 min,"Animation, Comedy, Family",Sid the Sloth is on Santa's naughty list.
3,s4,Movie,The Queen Family Singalong,Hamish Hamilton,"Darren Criss, Adam Lambert, Derek Hough, Alexa...",NaN,"November 26, 2021",2021,TV-PG,41 min,Musical,"This is real life, not just fantasy!"
4,s5,TV Show,The Beatles: Get Back,NaN,"John Lennon, Paul McCartney, George Harrison, ...",NaN,"November 25, 2021",2021,NaN,1 Season,"Docuseries, Historical, Music",A three-part documentary from Peter Jackson ca...


In [7]:
movie_data = pd.concat([netflix,amazon,hulu,disney],ignore_index=True)
movie_data

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...,...,...,...,...,...
22993,s1446,Movie,X-Men Origins: Wolverine,Gavin Hood,"Hugh Jackman, Liev Schreiber, Danny Huston, wi...","United States, United Kingdom","June 4, 2021",2009,PG-13,108 min,"Action-Adventure, Family, Science Fiction",Wolverine unites with legendary X-Men to fight...
22994,s1447,Movie,Night at the Museum: Battle of the Smithsonian,Shawn Levy,"Ben Stiller, Amy Adams, Owen Wilson, Hank Azar...","United States, Canada","April 2, 2021",2009,PG,106 min,"Action-Adventure, Comedy, Family",Larry Daley returns to rescue some old friends...
22995,s1448,Movie,Eddie the Eagle,Dexter Fletcher,"Tom Costello, Jo Hartley, Keith Allen, Dickon ...","United Kingdom, Germany, United States","December 18, 2020",2016,PG-13,107 min,"Biographical, Comedy, Drama","True story of Eddie Edwards, a British ski-jum..."
22996,s1449,Movie,Bend It Like Beckham,Gurinder Chadha,"Parminder Nagra, Keira Knightley, Jonathan Rhy...","United Kingdom, Germany, United States","September 18, 2020",2003,PG-13,112 min,"Buddy, Comedy, Coming of Age",Despite the wishes of their traditional famili...


In [8]:
movie_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22998 entries, 0 to 22997
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       22998 non-null  object
 1   type          22998 non-null  object
 2   title         22998 non-null  object
 3   director      14739 non-null  object
 4   cast          17677 non-null  object
 5   country       11499 non-null  object
 6   date_added    13444 non-null  object
 7   release_year  22998 non-null  int64 
 8   rating        22134 non-null  object
 9   duration      22516 non-null  object
 10  listed_in     22998 non-null  object
 11  description   22994 non-null  object
dtypes: int64(1), object(11)
memory usage: 2.1+ MB


In [9]:
movie_data.isnull().sum()

show_id             0
type                0
title               0
director         8259
cast             5321
country         11499
date_added       9554
release_year        0
rating            864
duration          482
listed_in           0
description         4
dtype: int64

In [10]:
## filling missing values
movie_data.fillna(' ',inplace=True)

In [11]:
movie_data.isnull().sum()

show_id         0
type            0
title           0
director        0
cast            0
country         0
date_added      0
release_year    0
rating          0
duration        0
listed_in       0
description     0
dtype: int64

In [12]:
# feature selection
movie_data['release_year'] = str(movie_data['release_year'])
selected_features = ['type','title','director','cast','country','release_year','rating','listed_in','description']

In [13]:
combined_feat =  movie_data['type'] + ' ' + movie_data['title'] + ' ' + movie_data['director'] + ' ' + movie_data['cast'] + ' ' + movie_data['country'] + ' ' + movie_data['release_year'] + ' ' + movie_data['rating'] + ' '+ movie_data['listed_in'] + ' ' + movie_data['description']

In [14]:
combined_feat

0        Movie Dick Johnson Is Dead Kirsten Johnson   U...
1        TV Show Blood & Water   Ama Qamata, Khosi Ngem...
2        TV Show Ganglands Julien Leclercq Sami Bouajil...
3        TV Show Jailbirds New Orleans       0        2...
4        TV Show Kota Factory   Mayur More, Jitendra Ku...
                               ...                        
22993    Movie X-Men Origins: Wolverine Gavin Hood Hugh...
22994    Movie Night at the Museum: Battle of the Smith...
22995    Movie Eddie the Eagle Dexter Fletcher Tom Cost...
22996    Movie Bend It Like Beckham Gurinder Chadha Par...
22997    Movie Captain Sparky vs. The Flying Saucers Ma...
Length: 22998, dtype: object

In [15]:
# perform feature transformation

vector = TfidfVectorizer()
feat_vector = vector.fit_transform(combined_feat)

In [16]:
print(feat_vector)

  (0, 35319)	0.23392675039587163
  (0, 24920)	0.14256008970385983
  (0, 10332)	0.14427054388573574
  (0, 75366)	0.11318346576946378
  (0, 32554)	0.12032723192813319
  (0, 76107)	0.042169320550409485
  (0, 81220)	0.1678462795031045
  (0, 15826)	0.22729248225348897
  (0, 3601)	0.04006758422099979
  (0, 35907)	0.23794288591587265
  (0, 35071)	0.04760996807100061
  (0, 18676)	0.1281085336551382
  (0, 71442)	0.2144735246732795
  (0, 26035)	0.1722614838191261
  (0, 43575)	0.08729598601809282
  (0, 33232)	0.12703302064849792
  (0, 54052)	0.04199238011902595
  (0, 23487)	0.14528552498381217
  (0, 75320)	0.07044114114872982
  (0, 52175)	0.24826310844897106
  (0, 25418)	0.11990568045594477
  (0, 32693)	0.07739484513547962
  (0, 5080)	0.0770991982991972
  (0, 20987)	0.10649858431799956
  (0, 60)	0.08269060748198927
  :	:
  (22997, 1661)	0.09720765470595935
  (22997, 1271)	0.0855536070424482
  (22997, 77682)	0.05118674963007425
  (22997, 54052)	0.045976085986753776
  (22997, 75320)	0.0385618528085

In [26]:
feat_vector

<22998x84670 sparse matrix of type '<class 'numpy.float64'>'
	with 1471189 stored elements in Compressed Sparse Row format>

## Cosine Similarity

Cosine similarity measures the similarity between two vectors of an inner product space. 

In [17]:
similarity = cosine_similarity(feat_vector)

In [18]:
print(similarity)

[[1.         0.02075008 0.03392957 ... 0.05022211 0.05220779 0.04492253]
 [0.02075008 1.         0.04120888 ... 0.01168297 0.01218688 0.022117  ]
 [0.03392957 0.04120888 1.         ... 0.01688655 0.0176149  0.03993927]
 ...
 [0.05022211 0.01168297 0.01688655 ... 1.         0.07503608 0.04112634]
 [0.05220779 0.01218688 0.0176149  ... 0.07503608 1.         0.04175601]
 [0.04492253 0.022117   0.03993927 ... 0.04112634 0.04175601 1.        ]]


## MOVIE RECOMMENDER SYSTEM

In [21]:
def movie_recommender():
    movie_name = input('ENTER MOVIE NAME')
    print('======================== YOU WOULD LOVE TO WATCH =====================')
    # get list of movies
    list_of_all_titles = movie_data['title'].tolist()
    # choose the closest name to the movie in the dataset
    close_movie = difflib.get_close_matches(movie_name,list_of_all_titles)
    movie_index = movie_data[movie_data['title']==close_movie[0]].index.values[0]
    similarity_score = list(enumerate(similarity[movie_index]))
    # sort based on similarity score
    sorted_score = sorted(similarity_score,key=lambda x:x[1],reverse=True)
    # get movie titles
    i=1
    for score in sorted_score :
        index = score[0]
        movie_title = movie_data[movie_data.index==index].title.values[0]
        if (i<21) :
            print(i, ' ' , movie_title )
            i+=1

In [25]:
movie_recommender()

ENTER MOVIE NAME king of boys


======================== YOU WOULD LOVE TO WATCH =====================
1   King of Boys
2   King of Boys: The Return of the King
3   The Wedding Party
4   The Royal Hibiscus Hotel
5   The Wedding Party 2: Destination Dubai
6   The Arbitration
7   Kambili: The Whole 30 Yards
8   Shadow Parties
9   10 Days in Sun City
10   Lara and the Beat
11   The Set Up
12   In Line
13   Toni Morrison: The Pieces I Am
14   The Last Word
15   Òlòtūré
16   2 Weeks in Lagos
17   Couple of Days
18   Crazy, Lovely, Cool
19   Love Is War
20   Merry Men: The Real Yoruba Demons
